# H3-H4 dimer with truncated tails in water
## Minimize and equilibrate
 - Minimize
 - Equilibrate
 
 Sanity checks along the way.
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

In [ ]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl

%matplotlib inline

folder="h3-h4_tm" #This one is important - set it to a unique value
scuser="shaytan_1609" #username on lomonosoc

In [ ]:
%%bash
rm -rf MDProtocols

In [ ]:
#Set protocols URL
prot_url="https://api.github.com/repos/intbio/gmx_protocols/contents/amber"

In [ ]:
get_files_from_git(prot_url,'MDProtocols/')

### 1_Minimization - steepest descent gradient method 10000 steps with POSRED constraints

#### Prepare tpr files using GROMPP
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-grompp.html

In [ ]:
%%bash
#Prepare system
mkdir -p GMX_run
gmx grompp -f MDProtocols/1_minim.mdp -c GMX_system/init_solv_ions.pdb -p GMX_system/topol.top \
-o GMX_run/1_minim.tpr -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_1_minim.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -nt 20 -deffnm 1_minim # if you want to run locally, nt is the number of threads

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='1_minim'

In [ ]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 1 -J $1 lomo2_run.sh $2"

In [ ]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

### 2_Equilibration 200 ps with postional restraints of 500 kJ/mol/A2

In [30]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/2_equil.mdp -c GMX_run/1_minim.gro -p GMX_system/topol.top \
-o GMX_run/2_equil.tpr -v  -r GMX_run/1_minim.gro -n GMX_system/index.ndx -po GMX_run/mdout_2_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K
Calculated rlist for 1x1 atom pair-list as 0.836 nm, buffer size 0.036 nm
Set rlist, assuming 4x4 atom pair-list, to 0.800 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 96x96x96, spacing 0.107 0.107 0.107
This run will generate roughly 15 Mb of data


                      :-) GROMACS - gmx grompp, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 2_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [31]:
deffnm='2_equil'

In [32]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [33]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 4 -J $1 lomo2_run.sh $2"

   Lomonosov-2 supercomputer welcomes you!




In [34]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

908768


In [35]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            906340   compute  jobname shaytan_ PD       0:00      4 (Dependency)
            906341   compute  jobname shaytan_ PD       0:00      4 (Dependency)
            906342   compute  jobname shaytan_ PD       0:00      4 (Dependency)
            906343   compute  jobname shaytan_ PD       0:00      4 (Dependency)
            906331   compute  jobname shaytan_  R   14:38:57      4 n[49729-49732]
            908059   compute  jobname shaytan_  R 2-02:38:31      4 n[51108,51314,51321,51703]
            908735   compute  jobname shaytan_  R    3:07:18      4 n[49127-49130]
            908768      test h3-h4_tm shaytan_  R       0:04      4 n[48616-48619]


In [36]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

Working dir:  /mnt/scratch/users/shaytan_1609/h3-h4_tm/GMX_run
Command line:
  gmx_mpi mdrun -ntomp 6 -gputasks 00 -pme cpu -nb gpu -deffnm 2_equil

Reading file 2_equil.tpr, VERSION 2018.3 (single precision)
The number of OpenMP threads was set by environment variable OMP_NUM_THREADS to 6 (and the command-line setting agreed with that)
Changing nstlist from 10 to 50, rlist from 0.8 to 0.925

Using 8 MPI processes
Using 6 OpenMP threads per MPI process

On host n48616.10p.parallel.ru 1 GPU user-selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 2 ranks on this node:
  PP:0,PP:0

NOTE: You assigned the same GPU ID(s) to multiple ranks, which is a good idea if you have measured the performance of alternatives.

NOTE: GROMACS was configured without NVML support hence it can not exploit
      application clocks of the detected Tesla K40s GPU to improve performance.
      Recompile with the NVML library (compatible with the driver used) or set application clocks manually.



In [38]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

   Energies (kJ/mol)
          Angle    Proper Dih.  Improper Dih.          LJ-14     Coulomb-14
    1.53936e+04    1.01015e+04    7.01417e+02    3.78760e+03    1.42372e+04
        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip. Position Rest.
    1.89077e+05   -9.81779e+03   -1.42180e+06    7.59299e+03    6.56798e+02
      Potential    Kinetic En.   Total Energy  Conserved En.    Temperature
   -1.19007e+06    1.85283e+05   -1.00479e+06   -1.39348e+06    2.75886e+02
 Pres. DC (bar) Pressure (bar)   Constr. rmsd
   -4.09439e+02   -1.21289e+02    1.79313e-06

DD  step 3999 load imb.: force  2.4%

           Step           Time
           4000        8.00000

   Energies (kJ/mol)
          Angle    Proper Dih.  Improper Dih.          LJ-14     Coulomb-14
    1.35569e+04    9.86309e+03    8.03559e+02    3.61378e+03    1.43689e+04
        LJ (SR)  Disper. corr.   Coulomb (SR)   Coul. recip. Position Rest.
    1.80609e+05   -9.77383e+03   -1.39235e+06    8.06307e+03    7.70154e+02
    

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

### 3_Equilibration 200 ps with postional restraints of 50 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/2_equil.mdp -c GMX_run/2_equil.gro -p GMX_system/topol.top \
-o GMX_run/3_equil.tpr -v  -r GMX_run/2_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_3_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 3_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='3_equil'

In [ ]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 4 -J $1 lomo2_run.sh $2"

In [ ]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

### 4_Equilibration 200 ps with postional restraints of 5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f Protocols/3_equil.mdp -c GMX_run/3_equil.gro -p GMX_system/topol.top \
-o GMX_run/4_equil.tpr -v  -r GMX_run/3_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_4_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 4_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='4_equil'

In [ ]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 4 -J $1 lomo2_run.sh $2"

In [ ]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

### 5_Equilibration 200 ps with postional restraints of 0.5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/4_equil.mdp -c GMX_run/4_equil.gro -p GMX_system/topol.top \
-o GMX_run/5_equil.tpr -v  -r GMX_run/4_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_5_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 5_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='5_equil'

In [ ]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 4 -J $1 lomo2_run.sh $2"

In [ ]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

### 6_Equilibration 200 ps unrestrained

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/5_equil.mdp -c GMX_run/5_equil.gro -p GMX_system/topol.top \
-o GMX_run/6_equil.tpr -v  -r GMX_run/5_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_6_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -deffnm 6_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='6_equil'

In [ ]:
%%bash -s "$folder" --err err
cp Run_scripts/lomo2_run.sh GMX_run/
ssh lomo2 "mkdir -p ~/_scratch/$1" && scp -r GMX_run lomo2:~/_scratch/$1/;

In [ ]:
%%bash -s "$folder" "$deffnm" --out out
ssh lomo2 "module load slurm gromacs/2018-gcc; cd ~/_scratch/$1/GMX_run; \
sbatch -p test -t 00:15:00 -N 4 -J $1 lomo2_run.sh $2"

In [ ]:
#get job id
job_idm=re.findall(r'\d+', out)[0]
print(job_idm)

In [ ]:
%%bash -s "$scuser"   --err err
# check job status
ssh lomo2 "module load slurm gromacs/2018-gcc; squeue -u $1 " || echo "No job in queue"

In [ ]:
%%bash -s "$folder"  "$job_idm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/egmx.$2"

In [ ]:
%%bash -s "$folder"  "$deffnm"  --err err
# check job status
ssh lomo2 "tail -n 25 ~/_scratch/$1/GMX_run/$2.log"

In [ ]:
%%bash -s "$folder" --err err 
#Purge # files and copy

ssh lomo2 "rm -f ~/_scratch/$1/GMX_run/\#*; " && scp lomo2:~/_scratch/$1/GMX_run/* GMX_run/ ;

#### Analyze what we get

In [ ]:
view_nucl('GMX_system/init_solv_ions.pdb','GMX_run/%s.trr'%deffnm)

As a result we have equlibrated system ready for production simulations.
We copy it to GMX_system.

In [ ]:
%%bash
cp GMX_run/6_equil.gro GMX_system/

#following files will be needed
#gmx grompp -f Protocols/7_prod.mdp -c GMX_system/6_equil.gro -p GMX_system/topol.top \
#-o GMX_run/7_prod.tpr -v  -r GMX_system/6_equil.gro -n GMX_system/index.ndx -po GMX_run/mdout_7_prod.mdp

# Saving final equilibrated results

The main aim of this repo is to produce equilibrated simulation systems, here we place into results folder the set of equilibrated files of step 6.


In [ ]:
%%bash
gmx trjconv -f GMX_run/6_equil.gro -s GMX_run/6_equil.tpr -o GMX_run/6_equil.pdb << !
0
!

In [ ]:
%%bash --out out
gmx trjconv -f GMX_system/init_solv_ions.pdb -s GMX_run/6_equil.tpr -o results/init_prot_xray.pdb << !
1
!


In [ ]:
%%bash
mkdir -p results
cp GMX_run/6_equil* results/